In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
import graphing
import preprocessing
from preprocessing import remove_columns, eliminar_filas_por_valor, unique_values, filter_values, suma_columna, remove_columns_with_nulls, eliminar_nulls
from excel_formulas import rrc_unidad, reserva_de_riesgo_en_curso, calcular_plazo, generar_devengado, generar_rrc_unidad

In [2]:
# Replace 'your_data.xlsx' with the actual path to your Excel file
file_path = 'PRODUCCION22-23.xlsx'
template_file_path = 'ProduccionAutomovil2022-edited.xlsx'

In [3]:
# Read the excel file
df_original = pd.read_excel(file_path)

df_template = pd.read_excel(template_file_path)

In [4]:
# Remove columns that are not needed
df = remove_columns(df_original, df_template)
# df.head()

In [5]:
# Generate excel file with the data
# df.to_excel('output.xlsx')

In [6]:
# Listado de productos a utilizar
valid_products = ['AUTOMOVIL- ALTA GAMA','PLAN ACCIONISTAS','REGIONAL','REGIONAL - ITAIPU / CONMEBOL','REGIONAL 0KM','REGIONAL MAX','REGIONAL PLUS','REGIONAL SUPERIOR']

# Utilizar solo los productos validos
df = filter_values(df, 'Nombre Producto', valid_products)

# Elininar filas de anulaciones
df = eliminar_filas_por_valor(df, 'Nombre Tipo Póliza', 'Anulacion')

In [ ]:
# Generate excel file with the data
# df.to_excel('output.xlsx')

In [7]:
# Show columns
df.columns

Index(['Nombre Tipo Póliza', 'F/Emisión', 'Fec. Desde', 'Fec. Hasta',
       'Nombre Producto', 'Fec. Desde Art.', 'Fec. Hasta Art.', 'Prima Art.'],
      dtype='object')

In [24]:
# Crear columna de Plazo
df['Plazo'] = df['Fec. Hasta Art.'] - df['Fec. Desde Art.']

In [25]:
# Show columns
print(df.columns)
print("Longitud original:", len(df))

Index(['Nombre Tipo Póliza', 'F/Emisión', 'Fec. Desde', 'Fec. Hasta',
       'Nombre Producto', 'Fec. Desde Art.', 'Fec. Hasta Art.', 'Prima Art.',
       'Plazo', 'Devengado', 'RRC Unidad'],
      dtype='object')
Longitud original: 179


In [ ]:
# Generate excel file with the data
# df.to_excel('output.xlsx')

In [26]:
from datetime import datetime, timedelta

# Crear variable inicio corte y fin corte en formato fecha
inicio_corte = '2022-07-01'
fin_corte = '2023-06-30'

# Convert 'inicio_corte' and 'fin_corte' to datetime objects
inicio_corte = pd.to_datetime(inicio_corte)
fin_corte = pd.to_datetime(fin_corte)

# Apply the function to create the 'Devengado' column
df['Devengado'] = df.apply(lambda row: generar_devengado(row, inicio_corte, fin_corte), axis=1)

In [27]:
# Show columns
print(df.columns)
print("Longitud original:", len(df))

Index(['Nombre Tipo Póliza', 'F/Emisión', 'Fec. Desde', 'Fec. Hasta',
       'Nombre Producto', 'Fec. Desde Art.', 'Fec. Hasta Art.', 'Prima Art.',
       'Plazo', 'Devengado', 'RRC Unidad'],
      dtype='object')
Longitud original: 179


In [ ]:
# Generate excel file with the data
# df.to_excel('output.xlsx')

In [30]:
# Show columns
df.columns

Index(['Nombre Tipo Póliza', 'F/Emisión', 'Fec. Desde', 'Fec. Hasta',
       'Nombre Producto', 'Fec. Desde Art.', 'Fec. Hasta Art.', 'Prima Art.',
       'Plazo', 'Devengado', 'RRC Unidad'],
      dtype='object')

In [29]:
# Crear columna RRC Unidad
df['RRC Unidad'] = np.where(df['Plazo'] > pd.Timedelta(0), df['Devengado'] / df['Plazo'].dt.days, "")

# Show columns
df.columns

Index(['Nombre Tipo Póliza', 'F/Emisión', 'Fec. Desde', 'Fec. Hasta',
       'Nombre Producto', 'Fec. Desde Art.', 'Fec. Hasta Art.', 'Prima Art.',
       'Plazo', 'Devengado', 'RRC Unidad'],
      dtype='object')

In [16]:
# Generate excel
# df.to_excel('output.xlsx')

In [ ]:
# Convert 'Plazo' to floats
df['Plazo'] = df['Plazo'].dt.days.astype(float)

# Convert 'RRC Unidad' to floats
df['RRC Unidad'] = df['RRC Unidad'].astype(float)


In [35]:

# Create column 'RRC'
df['RRC'] = np.where((df['Plazo'] > 0) & (df['RRC Unidad'] > 0), df['RRC Unidad'] * df['Prima Art.'], 0)

# Show columns
df.columns

Index(['Nombre Tipo Póliza', 'F/Emisión', 'Fec. Desde', 'Fec. Hasta',
       'Nombre Producto', 'Fec. Desde Art.', 'Fec. Hasta Art.', 'Prima Art.',
       'Plazo', 'Devengado', 'RRC Unidad', 'RRC'],
      dtype='object')

In [36]:
# Generate excel
df.to_excel('output.xlsx')

In [37]:
# Sumar la columna RRC
suma_rrc = suma_columna(df,'RRC')
print('Suma RRC: ',suma_rrc)

Suma RRC:  555928737.3972945


In [38]:
# Return excel file
df.to_excel('ResultadoProduccion.xlsx', index=False)
print("Longitud:", len(df))

Longitud: 179


In [39]:
# Contar las filas donde los valores de la columna RRC no sean cero
df_rrc = df[df['RRC'] != 0]
print("Longitud:", len(df_rrc))

Longitud: 161
